In [94]:
from flask import Flask, request, jsonify, render_template
from heap import MinHeap
import copy

app = Flask(__name__)

class PrefixTreeNode:
    def __init__(self,string = None, json = None, frequency = 1):
        self.json = json
        self.string = string
        self.frequency = frequency
    def __lt__(self, other):
        if self.frequency < other.frequency:
            return True
        return False        
    def __gt__(self, other):
        if self.frequency > other.frequency:
            return True
        return False 
    def __le__(self, other):
        if self.frequency <= other.frequency:
            return True
        return False        
    def __ge__(self, other):
        if self.frequency >= other.frequency:
            return True
        return False
class PrefixTree:
    #TODO реализация класса prefix tree, методы как на лекции + метод дать топ 10 продолжений. Скажем на строку кросс выдаем кроссовки, кроссовочки итп. Как хранить топ? 
    #Решать вам. Можно, конечно, обходить все ноды, но это долго. Дешевле чуток проиграть по памяти, зато отдавать быстро (скажем можно взять кучу)
    #В терминальных (конечных) нодах может лежать json с топ актерами.
    def __init__(self):
        self.root = [{}]
        
    def add(self, string = None, json = None, frequency = 1):
        string = string.lower()
        if self.check(string):
            return
        wrk_dict = self.root        
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                wrk_dict[0][i] = [{}, MinHeap([])]
                wrk_dict = wrk_dict[0][i]
            node = PrefixTreeNode(string, json, frequency)
            if len(wrk_dict[1].array) < 10:
                wrk_dict[1].add(node)
            else:
                min_node = wrk_dict[1].extract_minimum()
                if node >= min_node:
                    wrk_dict[1].add(node)
                else:
                    wrk_dict[1].add(min_node)
        wrk_dict.append(json)        
        
    def check(self, string):
        string = string.lower()
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                return False
        if len(wrk_dict) == 3:
            return True
        return False
    
    def check_part(self, string):
        string = string.lower()
        wrk_dict = self.root
        for i in string:
            if i in wrk_dict[0]:
                wrk_dict = wrk_dict[0][i]
            else:
                return False
        return True
    
    def top10(self, string):
        string = string.lower()
        top = []
        if self.check_part(string):
            wrk_dict = self.root
            for i in string:
                wrk_dict = wrk_dict[0][i]
            node_array = wrk_dict[1].array.copy
            node_array.sort(reverse = True)
            for i in node_array:
                suggest = {'suggest': s.string, 'json': s.json}
                top.append(suggest)
                print (top)
        return top
        

def init_prefix_tree(filename):
    #TODO в данном методе загружаем данные из файла. Предположим вормат файла "Строка, чтобы положить в дерево" \t "json значение для ноды" \t частота встречаемости
    f = open(filename, 'r')
    for line in f:
        data = line.split('\t')
        PrefixTree.add(data[0], data[1], data[2])
    f.close()

In [83]:
import pandas as pd
data = pd.read_pickle('parser_data.pkl')
df_pkl = pd.DataFrame(data)
title = []
title = df_pkl.Title

In [93]:
pTree = PrefixTree()
for i in title:
    pTree.add(i)
print (pTree.root[0])
print (pTree.check_part('й'))
print (pTree.top10('й'))

{'й': [{'о': [{'з': [{'е': [{'ф': [{'о': [{'в': [{}, <heap.MinHeap object at 0x7fba63c426a0>, None]}, <heap.MinHeap object at 0x7fba56427da0>]}, <heap.MinHeap object at 0x7fba56427390>]}, <heap.MinHeap object at 0x7fba56427a90>]}, <heap.MinHeap object at 0x7fba56427780>]}, <heap.MinHeap object at 0x7fba56427198>]}, <heap.MinHeap object at 0x7fba56427630>], 'е': [{'л': [{'и': [{'с': [{'е': [{'й': [{'с': [{'к': [{'и': [{'е': [{' ': [{'п': [{'о': [{'л': [{'я': [{}, <heap.MinHeap object at 0x7fba5828b048>, None]}, <heap.MinHeap object at 0x7fba5828b908>]}, <heap.MinHeap object at 0x7fba5828bac8>]}, <heap.MinHeap object at 0x7fba5828b940>]}, <heap.MinHeap object at 0x7fba5828b5c0>]}, <heap.MinHeap object at 0x7fba5828bba8>]}, <heap.MinHeap object at 0x7fba5828bda0>]}, <heap.MinHeap object at 0x7fba5828b898>]}, <heap.MinHeap object at 0x7fba5828bdd8>]}, <heap.MinHeap object at 0x7fba5828b710>]}, <heap.MinHeap object at 0x7fba5828be48>]}, <heap.MinHeap object at 0x7fba5828b0b8>]}, <heap.MinHe

TypeError: 'module' object is not callable

In [78]:
@app.route("/get_sudgest/<string>", methods=['GET', 'POST'])
def return_sudgest(string):
    #TODO по запросу string вернуть json, c топ-10 саджестами, и значениями из нод
    if request.method == 'GET':
        data = pTree.top10(string)
        return jsonify(data)    

@app.route("/")
def hello():
    #TODO должна возвращатьс инструкция по работе с сервером
    return render_template('hello.html')

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2018 22:29:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 22:29:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Nov/2018 22:29:34] "GET /get_sudgest/%D0%B0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 22:30:56] "GET /get_sudgest/ HTTP/1.1" 404 -
127.0.0.1 - - [13/Nov/2018 22:30:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [13/Nov/2018 22:30:59] "GET /get_sudgest/ak HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 22:31:06] "GET /get_sudgest/%D0%B0%D1%84%D1%84 HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 22:31:14] "GET /get_sudgest/%D0%B5%D0%B3%D0%B8%D0%BF%D0%B5%D1%82%D1%81%D0%BA%D0%B5%D0%B8 HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2018 22:31:18] "GET /get_sudgest/%D0%B5%D0%B3%D0%B8%D0%BF%D0%B5%D1%82%D1%81%D0%BA%D0%B8%D0%B5 HTTP/1.1" 200 -
